In [6]:
%matplotlib inline
import sys
sys.path.append('../../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from collections import OrderedDict, namedtuple, defaultdict
import time
#from modules.backtesting import BackTester
from modules.db_manager import open_file, product_info

In [489]:
feed = open_file('h5py', 'etc/ohlct.h5', mode='r')

In [496]:
class Market:
    """
    시장 정보 제공
    매매신호 생성기능
    """
    
    long = Long = L = 1
    short = Short = S = -1
    commission = 3.5 #편도 수수료
    
    def __init__(self, feed, sig_gen=None):
        """
        feed: pandas dataframe 형식의 시장 기초데이터
         1) 일별 date, open, high, low, close 가격정보

        sig_gen: signal 생성 함수
        """
        if not sig_gen:
            sig_gen = self.default_sig_gen
        self.pinfo = product_info()
        self.preprocessing(feed, sig_gen)
        
    def preprocessing(self, feed, sig_gen):
        """
        종목별로 시그널을 생성하여 feed에 merge하고
        종목별 데이터를 날짜순으로 모두 합침
        """
        header = feed.attrs['columns'].split(';')
        
        container = []
        for inst in feed.values():
            symbol =  inst.attrs['symbol']
            
            if symbol == 'None' or not symbol:
                continue
            datatable = pd.DataFrame(inst.value[:,1:], index=inst.value[:,0].astype('M8[s]'), columns=header[1:])
            datatable.sort_index(inplace=True)
            
            if sig_gen:
                sig_gen(datatable)
            columns = datatable.columns.tolist()
            new_column = [[symbol for i in range(len(columns))], columns]
            datatable.columns = new_column
            container.append(datatable)
        self.feed = pd.concat(container, axis=1).sort_index(axis=1)
    
    @classmethod
    def price_to_money(cls, value, pinfo):
        return value * pinfo['tick_value'] / pinfo['tick_unit']
    
    @classmethod
    def get_profit(self, pinfo, position, entryprice, exitprice, lot=1):
        """
        틱: (청산가격 - 진입가격)/틱단위
        손익계산: 랏수 * 틱가치 * 틱      
        """
        if np.isnan(entryprice) or np.isnan(exitprice):
            raise ValueError('Nan value can not be calculated')
        
        tick = round(position * (exitprice - entryprice)/pinfo['tick_unit'])
        profit = lot * pinfo['tick_value']* tick
        
        return profit, tick
    
    @classmethod
    def get_price(cls, pinfo, price1, price2, skid):
        bound = (price2 - price1)*skid
        price = np.random.uniform(price1, price1 + bound)
        price = round(price, pinfo['decimal_places'])
        return price
    
    
    @classmethod
    def get_lot(cls, pinfo, risk, risk_capa):
        lot = int(risk_capa / risk)
        return lot
    
    @classmethod
    def set_ATR(cls, metrics, span=20):
            df = pd.DataFrame()
            df['hl'] = metrics['high'] - metrics['low']
            df['hc'] = np.abs(metrics['high'] - metrics['close'].shift(1))
            df['lc'] = np.abs(metrics['low'] - metrics['close'].shift(1))
            df['TR'] = df.max(axis=1)
            metrics['ATR'] = df['TR'].ewm(span).mean()
    
    @staticmethod
    def default_sig_gen(datatable):
        """
        시장 기초데이터로부터 MA, trend index등을 생성하는 
        data preprocessing method이다.
        
        datatable: 종목별 ohlc 데이터를 저장한 pandas dataframe
        default 값으로 20일 이평과 60일 이평을 추가한다.
        
        """
        Market.set_ATR(datatable, span=20)
        
        datatable['ma20'] = datatable['close'].rolling(20).mean()
        datatable['ma60'] = datatable['close'].rolling(20).mean()
        datatable.dropna(inplace=True)
    

In [607]:
class Trader:
    """
    트레이더 봇
    매매전략 
    매매기록
    매매분석
    """
    def __init__(self, market, principal, portfolio_risk_ratio, risk_ratio, strategy=None):
        # 시장 정보 설정
        self.market = market
        self.pinfo = market.pinfo.copy()
        for inst in self.pinfo.values():
            # 보유중인 매매 저장
            inst['open_entries'] = {}
        
        if strategy:
            self.strategy = strategy
        else:
            self.strategy = self.default_strategy
    
        #매매관련기록
        self._equitylog = []
        self._tradelog = []
        
        #자산 초기값 설정
        self.capital = principal
        self.equity = principal
        self.avail_equity = principal
        self.real_equity = principal
        self.cum_commission = 0 #누적 수수료
        
        #매매규칙 초기값 설정
        self._portfolio_risk_ratio = portfolio_risk_ratio
        self._risk_ratio = risk_ratio
        
        #매매관련 정보
        self.entryid = 0
        self.open_entries = defaultdict(dict)
        self._tradelog = []
        self._equitylog = []
        self._rejected_order = []
    
    
    @property
    def tradelog(self):
        first_header = ['info','info','info','entry','entry','entry',
                        'exit','exit','exit','exit','result','result'] 
        second_header = ['id','inst','pos','date','price','lot',
                   'date','price','lot','forced','profit','tick']
        df = pd.DataFrame(self._tradelog)
        columns = [first_header, second_header]
        df.columns = columns
        return df
        #[entryid, inst, position, entrydate, entryprice, entrylot,
        #                       exitdate, exitprice, exitlot, profit, tick, force]
    @property
    def equitylog(self):
        columns = ['date', 'capital','open profit','equity','real equity',
                   'port risk', 'cum commission']
        return pd.DataFrame(self._equitylog, columns=columns).set_index('date')
    
    @property
    def rejected_order(self):
        columns = ['num trades','name','date','real equity', 'risk','risk capa',
                   'port risk', 'port risk capa','strategy','type']
        return pd.DataFrame(self._rejected_order, columns=columns)
    
    def risk_capa(self):
        return self._risk_ratio * self.real_equity
    
    def portfolio_risk_capa(self):
        return self._portfolio_risk_ratio * self.real_equity
    
    def portfolio_risk(self):
        portfolio_risk = 0
        for inst in trader.open_entries.values():
            for strat in inst.values():
                portfolio_risk += strat['risk']
                
        return portfolio_risk
    
    def get_risk(self, info, position, entryprice, stopprice):
        pricediff = position * (entryprice - stopprice)
        return Market.price_to_money(pricediff, info)
    
    def run_trade(self):
        symbols = self.market.feed.columns.levels[0] #종목 코드
        for date, metrics in self.market.feed.head(500).iterrows():
            print("\r now trading at %s         "%date, end='', flush=True)
            self.commission = 0
            for symbol in symbols:
                metric = metrics[symbol]
                info = self.pinfo[symbol]
                self.strategy(date, info, metric)
                
            # 강제청산 모니터링 및 매매기록
            self.force_stop(date, metrics)
            self.write_equitylog(date, metrics)
            if self.equity < 0:
                print("\nYou went bankrupt!")
                break
                
    def buy(self, info, date, position, entryprice, stopprice, strat='strat_0'):
        
        risk = self.get_risk(info, position, entryprice, stopprice)
        risk_capa = self.risk_capa()
        port_risk = self.portfolio_risk()
        port_risk_capa = self.portfolio_risk_capa()
        num_trades = self.count_trades()
        
        
        if risk > risk_capa:
            self._rejected_order.append([num_trades, info['name'], date, self.real_equity, risk, risk_capa,
                                         port_risk, port_risk_capa, strat, 1])
        
        elif port_risk + risk > port_risk_capa:
            self._rejected_order.append([num_trades, info['name'], date, self.real_equity, risk, risk_capa,
                                         port_risk, port_risk_capa, strat, 2])
        
        else:
            self.entryid += 1
            entrylot = Market.get_lot(info, risk, self.risk_capa())
            
            if strat in self.open_entries[info['group']].keys():
                raise AttributeError(f"open entry '{strat}' already exist!")
            
            else:
                if np.isnan(entryprice):
                    raise ValueError(f"Price {price} can not be NaN value")
                    
                symbol = info['group']
                self.open_entries[symbol][strat] = {
                    'entryid': self.entryid,
                    'strategy': strat,
                    'instrument': info,
                    'position': position,
                    'entrydate': date,
                    'entryprice': entryprice,
                    'entrylot': entrylot,
                    'openlot': entrylot,
                    'stopprice': stopprice,
                    'risk': risk
                }
                self.commission += (entrylot * Market.commission)
            

    def sell(self, info, date, price, strat='strat_0', force=False):
        symbol = info['group']
        if strat not in self.open_entries[symbol].keys():
            return
            #raise AttributeError(f"open entry '{strat}' does NOT exist!")
        
        else:
            open_entry = self.open_entries[symbol][strat]
        
        entryid = open_entry['entryid']
        inst = info['name']
        position = open_entry['position']
        entrydate = open_entry['entrydate']
        entryprice = open_entry['entryprice']
        entrylot = open_entry['entrylot']
        openlot = open_entry['openlot']
            
        exitdate = date
        exitprice = price
        exitlot = entrylot
            
        profit, tick =  Market.get_profit(info, position, entryprice, exitprice, lot=exitlot)
        self.commission += (exitlot * Market.commission)
        openlot = openlot - exitlot
            
        if openlot < 0:
            raise ValueError("exit lot cannot be greater than open lot")
                
        elif openlot == 0:
            del self.open_entries[symbol][strat]
            
        else:
            open_entry['openlot'] = openlot
                
            
        self._tradelog.append([entryid, inst, position, entrydate, entryprice, entrylot,
                               exitdate, exitprice, exitlot,force, profit, tick])
        self.capital += profit
        
    def write_equitylog(self, date, metrics):
        
        open_profit, open_margin = self.update_status(metrics)
        portfolio_risk = self.portfolio_risk()
        
        self.cum_commission += self.commission
        self.capital = self.capital - self.commission
        self.equity = self.capital + open_profit
        self.avail_equity = self.equity - open_margin
        self.real_equity = self.equity - portfolio_risk
        self._equitylog.append([date, self.capital, open_profit, self.equity,
                                self.real_equity, portfolio_risk,  self.cum_commission])
        
    def update_status(self, metrics):
        """
        1. stop price 업데이트
        2. 자산 업데이트
        """
        
        open_profit = 0
        open_margin = 0
        
        for inst in trader.open_entries.values():
            for strat in inst.values():
                info = strat['instrument']
                symbol = info['group']
                metric = metrics[symbol]
                
                if not np.isnan(metric['close']):
                    profit, tick = Market.get_profit(info, strat['position'], strat['entryprice'],
                                                     metric['close'], lot=strat['openlot'])
                
                    strat['open_profit'] = profit
                    
                open_profit += strat['open_profit']
                open_margin += strat['openlot'] * info['keep_margin']
                
                if np.isnan(open_profit):
                    raise ValueError("Open profit can not be NaN value")
                    
                #stop price and risk update
                stopprice = self.stop_rule(info, metric)
                
                if not np.isnan(stopprice):
                    risk = self.get_risk(info, strat['position'], strat['entryprice'], stopprice)
                    strat['stopprice'] = stopprice
                    if risk < 0:
                        strat['risk'] = 0
                    
                    else:
                        strat['risk'] = risk
                    
        return open_profit, open_margin
        
    def force_stop(self, date, metrics):
        for inst in trader.open_entries.values():
            for key in list(inst):
                strat = inst[key]
                info = strat['instrument']
                symbol = info['group']
                high = metrics[symbol]['high']
                low = metrics[symbol]['low']
                
                if (not np.isnan(high)) and (not np.isnan(low)) :
                    position = strat['position']
                    stopprice = strat['stopprice']
                    if position == Market.long:
                        pricediff = stopprice - low
                    elif position == Market.short:
                        pricediff = high - stopprice
                    else:
                        raise ValueError("unexcpected position value")
                    
                    if pricediff > 0:
                        self.sell(info, date, stopprice, strat=strat['strategy'], force=True)
    
    def count_trades(self):
        cnt = 0
        for i in self.open_entries.values():
            for k in i.values():
                cnt += 1
        return cnt
    
    def default_strategy(trader, date, info, metric):
        """
        ** default strategy: long only MA cross system **
        진입: 20종가이평이 60종가이평 돌파상승 한 다음날 시가 진입
        청산: 진입한 투자금(증거금) 대비 10% 이상 평가손실시 청산
        """
        # 신호 확인
        if metric['signal'] == 1:
            #stop_price = Market.convert(metric['ATR'] * 5, info, origin='price', to='money')
            entryprice= Market.get_price(info, metric['open'], metric['high'], skid=0.5)
            info['refprice'] = entryprice
            stopprice = trader.stop_rule(info, metric)
            position = Market.long
            
            trader.buy(info, date, position, entryprice, stopprice, strat='TF')
        if metric['signal'] == -1:
            exitprice = Market.get_price(info, metric['open'], metric['low'], skid=0.5)
            trader.sell(info, date, exitprice, strat='TF')
            
    @staticmethod        
    def stop_rule(info, metric):
        stopprice = info['refprice'] - metric['ATR']*3
        return stopprice

In [576]:
def sig_gen(inst):
    #Average True Range
    Market.set_ATR(inst, span=20)
    
    ma20 = inst['close'].rolling(20).mean()
    ma60 = inst['close'].rolling(60).mean()
    sig = (ma20>ma60).astype('int')
    inst['signal'] = sig.diff().shift(1)
    inst['last_close'] = inst['close'].shift(1)
    inst.dropna(inplace=True)
    

In [608]:
market = Market(feed=feed, sig_gen=sig_gen)

In [613]:
trader = Trader(market=market,
                principal=100000,
                portfolio_risk_ratio=0.2,
                risk_ratio=0.05)

In [614]:
%%time
trader.run_trade()

 now trading at 2007-02-05 00:00:00         Wall time: 16.3 s
